<h1> Pydantic Basics: Creating and Using Models</h1>

Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.



### Example of using dataclass

In python, we can use the dataclasses to define the schema of an object. However, it doesn't enforce the schema. User can pass the value in different datatype than what was defined in the data class

In [1]:
from dataclasses import dataclass

# Use @dataclass decorator to convert a regular class to dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

In [2]:
person=Person(name="Krish",age=35,city="Bangalore")
print(person)

Person(name='Krish', age=35, city='Bangalore')


In [3]:
person=Person(name="Krish",age=35,city=35)
print(person)

Person(name='Krish', age=35, city=35)


<p style="background:#FFFDD0">☝️ Here we can see that code didn't perform the validation on the datatype.</p>

### Example of Pydantic Class

Pydantic enforces the schema validation

In [1]:
from pydantic import BaseModel

In [2]:
class Person1(BaseModel):
    name:str
    age:int
    city:str

person=Person1(name="Krish",age=35,city="Bangalore")
print(person)

name='Krish' age=35 city='Bangalore'


In [22]:
# Using pydantic, the code throws errror if schema is not followed
try:
    person1=Person1(name="Krish",age=35,city=35)

except Exception as e:
    print("❌ Validation Error", str(e))

❌ Validation Error 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


In [6]:
person2 = Person1(name="Krish",age=35,city="35")
print(person2)

name='Krish' age=35 city='35'


#### 1. Adding Optional Fields
Add optional fields using Python's Optional type:



In [7]:
from typing import Optional

class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary: Optional[float] = None #Optional with default value
    is_active: Optional[bool] = True #optional field with default value

In [8]:
emp1=Employee(id=1,name="John",department="CS")
print(emp1)

id=1 name='John' department='CS' salary=None is_active=True


In [9]:
emp2=Employee(id=2,name="Krish",department="CS",salary="30000")
print(emp2)

id=2 name='Krish' department='CS' salary=30000.0 is_active=True


Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided

- Pydantic will try to automatically convert the datatype if possible. Example, a string "3000" gets converted to 3000.00 as float in above code.

In [14]:
emp3=Employee(id=2,name="Krish",department="CS",salary="30000",is_active=1)
print(emp3)

id=2 name='Krish' department='CS' salary=30000.0 is_active=True


#### 2. Adding type for List of Strings
Add optional fields using Python's Optional type:

In [11]:
from typing import List

class Classroom(BaseModel):
    room_number:str
    students: List[str] #List of strings
    capacity:int

In [11]:
# Create a classroom
classroom = Classroom(
    room_number="A101",
    students=("Alice", "Bob", "Charlie"),
    capacity=30
)
print(classroom)

room_number='A101' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [21]:
try:
    invalid_val = Classroom(
        room_number="A1",
        students=["Krish",123],
        capacity=30)

except Exception as e:
    print("❌",e)

❌ 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


<p style="background:#FFFDD0">☝️ Student accepts List of string, but the list contain one value as integer, which resulted in the error.</p>

#### 3. Model with Nested Models
Create complex structures with nested models:

In [16]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:str

class Customer(BaseModel):
    customer_id:int
    name:str
    address: Address  ## Nested Model

In [16]:
customer=Customer(customer_id=1,name="Krish",
                  address={"street":"Main street","city":"Boston","zip_code":"02108"})

print(customer)

customer_id=1 name='Krish' address=Address(street='Main street', city='Boston', zip_code='02108')


In [19]:
try:
    customer=Customer(customer_id=1,name="Krish",
                      address={"street":"Main street","city":123,"zip_code":"02108"})

except Exception as e:
    print("❌ ERROR DETAILS:", e)

❌ ERROR DETAILS: 1 validation error for Customer
address.city
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


#### 4. Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [27]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str = Field(min_length=2,max_length=50)
    price:float = Field(gt=0,le=10000)  ## greater than 0 and less than or equal to 1000
    quantity:int = Field(ge=0)

try:
    item=Item(name="Book", price=100000,quantity=10)
    print(item)
except Exception as e:
    print("❌ Error Details: ", e)

❌ Error Details:  1 validation error for Item
price
  Input should be less than or equal to 10000 [type=less_than_equal, input_value=100000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal


In [28]:
class User(BaseModel):
    username:str = Field(description="Unique username for the user")
    age:int = Field(default=18,description="User age default to 18 ")
    email:str = Field(default_factory=lambda: "user@example.com",description="Default email address")


# Examples
user1 = User(username="alice")
print(user1)

username='alice' age=18 email='user@example.com'


In [24]:
user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)

username='bob' age=25 email='bob@domain.com'


#### 5. Printing the Schema of Pydantic Class

We can print the schema of a pydantic object. It is very useful with LLM prompt templates. 

In [25]:
User.model_json_schema()

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User',
 'type': 'object'}

#### 6. Using Pydantic Logfire for tracking the logs


In [29]:
import logfire

In [30]:
from datetime import datetime
import logfire

from pydantic import BaseModel

logfire.configure(token='pylf_**********************************5')
logfire.info('Hello, Devendra!', place='World')

logfire.instrument_pydantic()  

class Delivery(BaseModel):
    timestamp: datetime
    dimensions: tuple[int, int]


# this will record details of a successful validation to logfire
m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])
print(repr(m.timestamp))

print(m.dimensions)

14:47:23.993 Hello, Devendra!
14:47:24.056 Pydantic Delivery validate_python


Logfire project URL: ]8;id=450432;https://logfire-us.pydantic.dev/devjakhmola1990/starter-project\https://logfire-us.pydantic.dev/devjakhmola1990/starter-project]8;;\

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(10, 20)


### More Exampls

In [31]:
from datetime import datetime
from pydantic import BaseModel, PositiveInt

class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

user = User(**external_data)  

print(user.id)  

print(user.model_dump())  


14:49:53.851 Pydantic User validate_python
123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


In [39]:
# continuing the above example...

from datetime import datetime
from pydantic import BaseModel, PositiveInt, ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 5, 'signup_ts': '2019-06-01 12:22', 'tastes': {}}  

try:
    user = User(**external_data) 
    print(user.model_dump())
except ValidationError as e:
    print(e)
    """
    [
        {
            'type': 'int_parsing',
            'loc': ('id',),
            'msg': 'Input should be a valid integer, unable to parse string as an integer',
            'input': 'not an int',
            'url': 'https://errors.pydantic.dev/2/v/int_parsing',
        },
        {
            'type': 'missing',
            'loc': ('signup_ts',),
            'msg': 'Field required',
            'input': {'id': 'not an int', 'tastes': {}},
            'url': 'https://errors.pydantic.dev/2/v/missing',
        },
    ]
    """

{'id': 5, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {}}


In [36]:
# Using Annoted and Literals

from typing import Annotated, Literal
from annotated_types import Gt
from pydantic import BaseModel

class Fruit(BaseModel):
    name: str  
    color: Literal['red', 'green']  # 👈 Only of these 2 values can be provided by the user
    weight: Annotated[float, Gt(0)]  
    bazam: dict[str, list[tuple[int, bool, float]]]  # 👈 Key as string and values as list of tuples
 
try:
    data = Fruit(
                name='Apple',
                color='k',
                weight= 10,
                bazam={'foobar': [(1, True, 10.2)]},
            )

    print(data)

    print(data.model_dump()) # Model structure as Dictionary

    print(data.model_json_schema()) # Model Structure as JSON
    
except Exception as e:
    print("❌ ERROR DETAILS:", e)
#> name='Apple' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}

14:56:13.146 Pydantic Fruit validate_python
❌ ERROR DETAILS: 1 validation error for Fruit
color
  Input should be 'red' or 'green' [type=literal_error, input_value='k', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error


In [43]:
from datetime import datetime

from pydantic import BaseModel, ValidationError

class Meeting(BaseModel):
    when: datetime
    where: str

# Model_validate allows to validate the schema

try:
    m = Meeting.model_validate(
        obj = {'when': '2020-01-01T12:00', 'where': 'home'}, 
        strict = True
    )
except ValidationError as e:
   print("❌ ERROR DETAILS:", e)

m_json = Meeting.model_validate_json(
    '{"when": "2020-01-01T12:00", "where": "home"}'
)
print("\n\n👈 JSON VALIDATE\n", m_json)
#> when=datetime.datetime(2020, 1, 1, 12, 0) where=b'home'

17:52:04.154 Pydantic Meeting validate_python
❌ ERROR DETAILS: 1 validation error for Meeting
when
  Input should be a valid datetime [type=datetime_type, input_value='2020-01-01T12:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/datetime_type
17:52:04.158 Pydantic Meeting validate_json


👈 JSON VALIDATE
 when=datetime.datetime(2020, 1, 1, 12, 0) where='home'


In [44]:
#!pip install pydantic[email]